In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import lightkurve as lk
from astropy.timeseries import LombScargle

from phi_m_alg import phi_m_radio as pmr

%matplotlib widget

In [ ]:
target_kepler_id = 'kic7917485'
instrument = 'kepler'
cadence = 'long'
author='kepler'

target = 'simulated'
amp1 = .001
amp2 = .01
orbital_freq = 1./(20*np.pi) #inverse days
phase_var = 0
mode1 = 15.3830026 # inverse days

lc_files = lk.search_lightcurve(target_kepler_id, cadence=cadence,author=author).download_all()
        
time, flux, flerr = np.array([]), np.array([]), np.array([])

for q,lc in enumerate(lc_files):
    lc = lc.remove_outliers(sigma=4)
    this_time = np.ma.array(lc['time'].value)

    ### to get rid of the masks ###
    this_flux = lc['pdcsap_flux'].value
    this_flerr = lc['pdcsap_flux_err'].value

    good = np.isfinite(this_time) & np.isfinite(this_flux) & np.isfinite(this_flerr)
    
    median_flux = np.nanmedian(this_flux)
    this_flux = this_flux[good] / median_flux
    this_flerr = this_flerr[good] / median_flux
    this_time = this_time[good]
    
    time = np.concatenate((time,this_time))
    flux = np.concatenate((flux,this_flux))
    flerr = np.concatenate((flerr,this_flerr))

flux=flux-1.

i = np.argsort(time)

ts, ys, flerrs = time[i],flux[i],flerr[i]

In [ ]:
### fake data ###

def make_flux(time, mode, orbital_freq, amp1, amp2, phi, flerr = None, additional_modes = None, additional_amps = None, additional_phis = None, gaps = None):
    """
    - gaps must be a list of slices to remove like: gaps = [(slice(2,5)),(slice(9,10))]
    """

    flux = amp1 * np.sin(2*np.pi * mode * (time - amp2 * np.sin(2 * np.pi * orbital_freq * time)) + phi)
    if flerr is not None:
        flux += np.ma.array(flerr) * np.random.normal(size=len(flerr))

    if additional_modes is not None:
        assert len(additional_modes) == len(additional_amps) == len(additional_phis) # must be equal lengths
        for freq_add,amp_add,phi_add in zip(additional_modes,additional_amps,additional_phis):
            flux += amp_add * np.sin(2*np.pi * freq_add * (time - amp2 * np.sin(2 * np.pi * orbital_freq * time)) + phi_add)

    if gaps is not None:
        flux = np.delete(flux,gaps)

    return flux-1

In [ ]:
flux_1mode = make_flux(ts, mode1, orbital_freq, amp1, amp2, phase_var, flerr = flerrs)

In [ ]:
qs = pmr.demodulate(ts,flux_1mode,mode1)

In [ ]:
fs, ps = pmr.listen(ts,qs,mode1)

In [ ]:
plt.figure()
plt.plot(fs,ps)
plt.axvline(orbital_freq,zorder=-10,c='pink')